<a href="https://colab.research.google.com/github/victorsmoreschi/Data-Studies/blob/main/Mobile_App_for_Lottery_Addiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Intro

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical [data](https://www.kaggle.com/datasets/datascienceai/lottery-dataset) coming from the national 6/49 lottery game in Canada. The data set has data for 3,665 drawings, dating from 1982 to 2018.

In [2]:
# Defining essential functions:

def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

def combination(n, k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    return numerator/denominator

## Basic Probability

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

In [3]:
def one_ticket_probability():
  C = combination(49, 6)
  result = (1 / C) * 100 # One winning probability
  message = 'Your probability of winning is: {:.10f}%'
  print(message.format(result))

one_ticket_probability()


Your probability of winning is: 0.0000071511%


## Database from Canada Lottery

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/649.csv')
df.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT          3665 non-null   int64 
 1   DRAW NUMBER      3665 non-null   int64 
 2   SEQUENCE NUMBER  3665 non-null   int64 
 3   DRAW DATE        3665 non-null   object
 4   NUMBER DRAWN 1   3665 non-null   int64 
 5   NUMBER DRAWN 2   3665 non-null   int64 
 6   NUMBER DRAWN 3   3665 non-null   int64 
 7   NUMBER DRAWN 4   3665 non-null   int64 
 8   NUMBER DRAWN 5   3665 non-null   int64 
 9   NUMBER DRAWN 6   3665 non-null   int64 
 10  BONUS NUMBER     3665 non-null   int64 
dtypes: int64(10), object(1)
memory usage: 315.1+ KB


In [6]:
def extract_numbers(row):
  numbers = row[4:10]
  numbers = set(numbers.values)
  return numbers

winning_numbers = df.apply(extract_numbers, axis=1)
winning_numbers.head()

,0
0,"{3, 41, 11, 12, 43, 14}"
1,"{33, 36, 37, 39, 8, 41}"
2,"{1, 6, 39, 23, 24, 27}"
3,"{3, 9, 10, 43, 13, 20}"
4,"{34, 5, 14, 47, 21, 31}"


Below. a function is created so the users try if the numbers they are going to pick won the lottery anytime since 1982. The idea is to show the user that is really hard to find a set that won.

In [7]:
def check_historical_ocurence(l, winning_numbers):
  s = set(l)
  check = s == winning_numbers
  print(sum(check))
  one_ticket_probability()

check_historical_ocurence([4,8,45,6,17,24], winning_numbers)

0
Your probability of winning is: 0.0000071511%


Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning —now, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

In [8]:
def multi_ticket_probability(n):
  C = combination(49,6)
  prop_of_w = n / C
  message = 'Your probability of winning is: {:.10f}%'
  print(message.format(prop_of_w * 100))

multi_ticket_probability(97)

Your probability of winning is: 0.0006936590%


We're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

In [9]:
def probability_less_6(n):

    n_combinations_ticket = combination(6, n)
    n_combinations_remaining = combination(43, 6 - n)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining

    n_combinations_total = combination(49, 6)
    probability = successful_outcomes / n_combinations_total

    probability_percentage = probability * 100
    combinations_simplified = round(n_combinations_total/successful_outcomes)
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n, probability_percentage,
                                                               int(combinations_simplified)))


In [10]:
probability_less_6(2)

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 8 chances to win.


In [12]:
# Probability of winning at least

def probability_at_least_k_winning(total_numbers, k_min):
    """Calculate the probability of having at least k_min winning numbers."""
    total_outcomes = combination(total_numbers, k_min)  # This is C(n, k_min)

    successful_outcomes = 0
    # Sum successful outcomes for having at least 2, 3, 4, or 5 winning numbers
    for k in range(k_min, 6):  # From k_min to 5 winning numbers
        successful_outcomes += combination(total_numbers, k)

    # Total possible outcomes (e.g., choosing 6 from total_numbers)
    total_possible_outcomes = combination(total_numbers, 6)  # Adjust this based on problem context

    # Calculate the probability
    probability = successful_outcomes / total_possible_outcomes
    return probability


probability_at_least_k_winning(60,4)

0.11883116883116883